In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: Vertex AI Forecasting Model Training Example

To use this Colaboratory notebook, you copy the notebook to your own Google Drive and open it with Colaboratory (or Colab). You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell. For more information about running notebooks in Colab, see the [Colab welcome page](https://colab.research.google.com/notebooks/welcome.ipynb).

This notebook demonstrates how to create an AutoML model based on a time series dataset. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

# Install Vertex AI SDK, Authenticate, and upload of a Dataset to your GCS bucket

After the SDK installation the kernel will be automatically restarted. You may see this error message `Your session crashed for an unknown reason` which is normal.

In [ ]:
%%capture
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
 
import IPython
 
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Enter your project and GCS bucket

Enter your Project ID in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

**If you don't know your project ID**, you may be able to get your project ID using gcloud.

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

You may also change the REGION variable, which is used for operations throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

The datasets we are using are samples from the [Iowa Liquor Retail Sales](https://pantheon.corp.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales) dataset. The training sample contains the sales from 2020 and the prediction sample (used in the batch prediction step) contains the January - April sales from 2021.

In [ ]:
TRAINING_DATASET_BQ_PATH = 'bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train'

# Initialize Vertex AI SDK

Initialize the *client* for Vertex AI.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# Create a Managed Time Series Dataset from BigQuery

This section will create a dataset from a BigQuery table.

In [ ]:
ds = aiplatform.datasets.TimeSeriesDataset.create(
    display_name='iowa_liquor_sales_train',
    bq_source=[TRAINING_DATASET_BQ_PATH])

ds.resource_name

# Launch a Training Job to Create a Model

Once we have defined your training script, we will create a model.

In [ ]:
time_column = "date"
time_series_identifier_column="store_name"
target_column="sale_dollars"

job = aiplatform.AutoMLForecastingTrainingJob(
    display_name='train-iowa-liquor-sales-automl_1',
    optimization_objective='minimize-rmse',    
    column_transformations=[
        {"timestamp": {"column_name": time_column}},
        {"numeric": {"column_name": target_column}},
        {"categorical": {"column_name": "city"}},
        {"categorical": {"column_name": "zip_code"}},
        {"categorical": {"column_name": "county"}},
    ]
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="iowa-liquor-sales-forecast-model", 
    predefined_split_column_name=None,
)

In [ ]:
#@title # Fetch Model Evaluation Metrics
#@markdown Fetch the model evaluation metrics calculated during training on the test set.

import pandas as pd

list_evaluation_pager = model.api_client.list_model_evaluations(parent=model.resource_name)
for model_evaluation in list_evaluation_pager:
  metrics_dict = {m[0]: m[1] for m in model_evaluation.metrics.items()}
  df = pd.DataFrame(metrics_dict.items(), columns=["Metric", "Value"])
  print(df.to_string(index=False))

# Run Batch Prediction

In [ ]:
#@markdown ## Create Output BigQuery Dataset
#@markdown First, create a new BigQuery dataset for the batch prediction output in the same region as the batch prediction input dataset. 

import os
from google.cloud import bigquery

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

batch_predict_bq_input_uri = "bq://bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict"
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions"
batch_predict_bq_output_dataset_path = "{}.{}".format(PROJECT_ID, batch_predict_bq_output_dataset_name)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(PROJECT_ID, batch_predict_bq_output_dataset_name)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client()
dataset = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_region = "US" # @param {type : "string"}
dataset.location = dataset_region
dataset = client.create_dataset(dataset)
print("Created bigquery dataset {} in {}".format(batch_predict_bq_output_dataset_path, dataset_region))

Run a batch prediction job to generate liquor sales forecasts for stores in Iowa from an input dataset containing historical sales.

In [ ]:
model.batch_predict(
   bigquery_source=batch_predict_bq_input_uri,
   instances_format="bigquery",
   bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
   predictions_format="bigquery",
   job_display_name="predict-iowa-liquor-sales-automl_1")

In [ ]:
#@title # Visualize the Forecasts
#@markdown Follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).

import urllib

tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
  if table.table_id.startswith(
      "predictions_") and table.table_id > prediction_table_id:
    prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id)


def _sanitize_bq_uri(bq_uri):
  if bq_uri.startswith("bq://"):
    bq_uri = bq_uri[5:]
  return bq_uri.replace(":", ".")


def get_data_studio_link(batch_prediction_bq_input_uri,
                         batch_prediction_bq_output_uri, time_column,
                         time_series_identifier_column, target_column):
  batch_prediction_bq_input_uri = _sanitize_bq_uri(
      batch_prediction_bq_input_uri)
  batch_prediction_bq_output_uri = _sanitize_bq_uri(
      batch_prediction_bq_output_uri)
  base_url = "https://datastudio.google.com/c/u/0/reporting"
  query = "SELECT \\n" \
  " CAST(input.{} as DATETIME) timestamp_col,\\n" \
  " CAST(input.{} as STRING) time_series_identifier_col,\\n" \
  " CAST(input.{} as NUMERIC) historical_values,\\n" \
  " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n" \
  " * \\n" \
  "FROM `{}` input\\n" \
  "LEFT JOIN `{}` output\\n" \
  "ON\\n" \
  "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n" \
  "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
  query = query.format(time_column, time_series_identifier_column,
                       target_column, target_column,
                       batch_prediction_bq_input_uri,
                       batch_prediction_bq_output_uri, time_column, time_column,
                       time_series_identifier_column,
                       time_series_identifier_column)
  params = {
      "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
      "ds0.connector": "BIG_QUERY",
      "ds0.projectId": PROJECT_ID,
      "ds0.billingProjectId": PROJECT_ID,
      "ds0.type": "CUSTOM_QUERY",
      "ds0.sql": query
  }
  params_str_parts = []
  for k, v in params.items():
    params_str_parts.append("\"{}\":\"{}\"".format(k, v))
  params_str = "".join(["{", ",".join(params_str_parts), "}"])
  return "{}?{}".format(base_url,
                        urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(batch_predict_bq_input_uri,
                         batch_predict_bq_output_uri, time_column,
                         time_series_identifier_column, target_column))


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:



In [ ]:
# Delete model resource
model.delete(sync=True)

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $BUCKET_NAME